# Configuring Amazon Bedrock boto3 Prerequisites

> *Complete all steps in us-west-1. 
Use this notebook with the  **`conda_tensorflow2_p310`** kernel in SageMaker Notebooks*

---
Make sure you have requested access for Claude 3 models in Bedrock.

After completing the steps, be sure to perform the clean up steps at the end to avoid unnecessary charges. 
Total cost should be a few $ if everything is terminated afterwards. Be sure to delete the SageMaker notebook at the end if no longer needed. 

---

### Install Required Libraries
Install required libraries like boto3, which is the AWS SDK for Python that interacts with Bedrock. And opensearch-py which is the Python client used to interact with OpenSearch.

In [1]:
%pip install -U opensearch-py==2.3.1
%pip install -U boto3==1.33.2
%pip install -U retrying==1.3.4

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 155.5 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.36.2
    Uninstalling botocore-1.36.2:
      Successfully uninstalled botocore-1.36.2
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.11.1
    Uninstalling s3transfer-0.11.1:
      Successfully uninstalled s3transfer-0.11.1
  Attempting uninstall: boto3
    Found existing installation: boto3 1.36.2
    Uninstalling boto3-1.36.2:
      Successfully uninstalled boto3-1.36.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.18.0 requires botocore<1.36.2,>=1.36.0, but you have botocore 1.33.13 which is incompatible.
awscli 1.37.2 requires botocore==1.36.2, but you have botocore 1.33.13 which is incompa

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Create the S3 Bucket
Import the required libraries, and set our variables.
Create an S3 bucket to store our custom data.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep
import random
from retrying import retry
suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
service = 'aoss'
s3_client = boto3.client('s3')
s3_suffix = f"{region_name}-faye-123"  # replace with your name and some random numbers.
bucket_name = f'bedrock-kb-{s3_suffix}' # replace it with your bucket name.
pp = pprint.PrettyPrinter(indent=2)

In [6]:
# Create S3 bucket for the knowledge base data source 
try:
    s3_client.head_bucket(Bucket=bucket_name)
    print(f'Bucket {bucket_name} Exists')
except ClientError as e:
    print(f'Bucket name:{bucket_name}')
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name,
    )

Bucket name:bedrock-kb-us-east-1-faye-123


### After the S3 Bucket is created, upload the data you want to use to the S3 bucket. You can use this dummy data: https://github.com/pluralsight-cloud/amazon-bedrock-building-genai-applications/blob/main/RAG-Demo/Human%20Resources%20Policy.docx

### Create the OSS Collection
Create the OpenSearch Serverless collection which is a container for OpenSearch indexes.

In [7]:
import boto3
import time
vector_store_name = f'bedrock-sample-rag-{suffix}'
index_name = f"bedrock-sample-rag-index-{suffix}"
aoss_client = boto3_session.client('opensearchserverless')
bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket_name)
bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']

In [8]:
# create security, network and data access policies for OSS
encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)
collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

In [9]:
# Get the OpenSearch serverless collection URL
collection_id = collection['createCollectionDetail']['id']
host = collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(host)

02ews76qmc68hnt0pfui.us-east-1.aoss.amazonaws.com


In [10]:
# Wait for collection creation to complete
# This can take a few minutes
response = aoss_client.batch_get_collection(names=[vector_store_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    interactive_sleep(30)
    response = aoss_client.batch_get_collection(names=[vector_store_name])
print('\nCollection successfully created:')
pp.pprint(response["collectionDetails"])

Creating collection...
Done!.........................

Collection successfully created:
[ { 'arn': 'arn:aws:aoss:us-east-1:548734566896:collection/02ews76qmc68hnt0pfui',
    'collectionEndpoint': 'https://02ews76qmc68hnt0pfui.us-east-1.aoss.amazonaws.com',
    'createdDate': 1738771594466,
    'dashboardEndpoint': 'https://02ews76qmc68hnt0pfui.us-east-1.aoss.amazonaws.com/_dashboards',
    'id': '02ews76qmc68hnt0pfui',
    'kmsKeyArn': 'auto',
    'lastModifiedDate': 1738771620993,
    'name': 'bedrock-sample-rag-499',
    'standbyReplicas': 'ENABLED',
    'status': 'ACTIVE',
    'type': 'VECTORSEARCH'}]


In [11]:
# create opensearch serverless access policy and attach it to Bedrock execution role
try:
    create_oss_policy_attach_bedrock_execution_role(collection_id=collection_id,
                                                    bedrock_kb_execution_role=bedrock_kb_execution_role)
    # It can take up to a minute for data access rules to be enforced
    interactive_sleep(60)
except Exception as e:
    print("Policy already exists")
    pp.pprint(e)

Opensearch serverless arn:  arn:aws:iam::548734566896:policy/AmazonBedrockOSSPolicyForKnowledgeBase_357
Done!.......................................................


### Create the OSS Vector Index
This will contain the vector embeddings, or numerical representations of our data. So that the Bedrock can make sense of our data, and understand the meaning it contains.

In [12]:
# Configure the OpenSearch Serverless vector index
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
credentials = boto3.Session().get_credentials()
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true",
       "number_of_shards": 1,
       "knn.algo_param.ef_search": 512,
       "number_of_replicas": 0,
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1536,
             "method": {
                 "name": "hnsw",
                 "engine": "faiss",
                 "space_type": "l2"
             },
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         }
      }
   }
}

# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [13]:
# Create index
try:
    response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
    print('\nCreating index:')
    pp.pprint(response)

    # index creation can take up to a minute
    interactive_sleep(60)
except RequestError as e:
    print(f'Error while trying to create the index, with error {e.error}\nyou may unmark the delete above to delete, and recreate the index')


Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-index-499',
  'shards_acknowledged': True}
Done!.......................................................


### Create The Knowledge Base and Ingest Data 
Configure the Bedrock Knowledge Base using the OpenSearch Serverless vector index. 
Data source will be our S3 bucket.

In [14]:
# After uploading your data to the private S3 bucket, create the Knowledge Base
opensearchServerlessConfiguration = {
            "collectionArn": collection["createCollectionDetail"]['arn'],
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

# Ingest strategy to ingest data from the data source
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# The data source to ingest documents from, into the OpenSearch serverless knowledge base index
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    
}

# The embedding model used by Bedrock to embed ingested documents, and real time prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v1"

name = f"bedrock-sample-knowledge-base-{suffix}"
description = "My custom data knowledge base."
roleArn = bedrock_kb_execution_role_arn

In [15]:
# Create a Knowledge Base
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = roleArn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [16]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [17]:
pp.pprint(kb)

{ 'createdAt': datetime.datetime(2025, 2, 5, 16, 18, 28, 627954, tzinfo=tzlocal()),
  'description': 'My custom data knowledge base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:548734566896:knowledge-base/RQ428EHTVD',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1'}},
  'knowledgeBaseId': 'RQ428EHTVD',
  'name': 'bedrock-sample-knowledge-base-499',
  'roleArn': 'arn:aws:iam::548734566896:role/AmazonBedrockExecutionRoleForKnowledgeBase_357',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-east-1:548734566896:collection/02ews76qmc68hnt0pfui',
                                                                   'fieldMapping': { 'metadataField': 'text-metadata',
                                                                                

In [18]:
# Get Knowledge Base 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [19]:
# Create a DataSource in Knowledge Base 
create_ds_response = bedrock_agent_client.create_data_source(
    name = name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
pp.pprint(ds)

{ 'createdAt': datetime.datetime(2025, 2, 5, 16, 18, 42, 178443, tzinfo=tzlocal()),
  'dataSourceConfiguration': { 's3Configuration': { 'bucketArn': 'arn:aws:s3:::bedrock-kb-us-east-1-faye-123'},
                               'type': 'S3'},
  'dataSourceId': '52QGFVQQ0S',
  'description': 'My custom data knowledge base.',
  'knowledgeBaseId': 'RQ428EHTVD',
  'name': 'bedrock-sample-knowledge-base-499',
  'status': 'AVAILABLE',
  'updatedAt': datetime.datetime(2025, 2, 5, 16, 18, 42, 178443, tzinfo=tzlocal()),
  'vectorIngestionConfiguration': { 'chunkingConfiguration': { 'chunkingStrategy': 'FIXED_SIZE',
                                                               'fixedSizeChunkingConfiguration': { 'maxTokens': 512,
                                                                                                   'overlapPercentage': 20}}}}


In [20]:
# Get Data Source 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

{'ResponseMetadata': {'RequestId': '0f76331a-5948-44f1-9454-4e75e048a199',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 05 Feb 2025 16:18:46 GMT',
   'content-type': 'application/json',
   'content-length': '588',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0f76331a-5948-44f1-9454-4e75e048a199',
   'x-amz-apigw-id': 'FhNYFEQ4IAMEu6g=',
   'x-amzn-trace-id': 'Root=1-67a38f66-7066b1c64cdd3a8d4040bef4'},
  'RetryAttempts': 0},
 'dataSource': {'knowledgeBaseId': 'RQ428EHTVD',
  'dataSourceId': '52QGFVQQ0S',
  'name': 'bedrock-sample-knowledge-base-499',
  'status': 'AVAILABLE',
  'description': 'My custom data knowledge base.',
  'dataSourceConfiguration': {'type': 'S3',
   's3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-kb-us-east-1-faye-123'}},
  'vectorIngestionConfiguration': {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE',
    'fixedSizeChunkingConfiguration': {'maxTokens': 512,
     'overlapPercentage': 20}}},
  'createdAt': datetime.datetime

In [21]:
# Start ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [22]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

{ 'dataSourceId': '52QGFVQQ0S',
  'ingestionJobId': 'VEZOZQSSJX',
  'knowledgeBaseId': 'RQ428EHTVD',
  'startedAt': datetime.datetime(2025, 2, 5, 16, 18, 53, 674512, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 2, 5, 16, 18, 53, 674512, tzinfo=tzlocal())}


In [23]:
# Get job 
while(job['status']!='COMPLETE' ):
    get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
    job = get_job_response["ingestionJob"]
pp.pprint(job)
interactive_sleep(40)

{ 'dataSourceId': '52QGFVQQ0S',
  'ingestionJobId': 'VEZOZQSSJX',
  'knowledgeBaseId': 'RQ428EHTVD',
  'startedAt': datetime.datetime(2025, 2, 5, 16, 18, 53, 674512, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 2, 5, 16, 18, 59, 655550, tzinfo=tzlocal())}
Done!...................................


In [24]:
# Print the knowledge base Id to use for invocation later
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)

'RQ428EHTVD'


In [25]:
# store the kb_id to use later in the invoke request
%store kb_id

Stored 'kb_id' (str)


### Test Using RetrieveAndGenerate API

In [26]:
# Test the Knowledge Base using the RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)
# Compare how two different Anthropic models respond
claude_model_ids = [ ["Claude 3 Sonnet", "anthropic.claude-3-sonnet-20240229-v1:0"], ["Claude Instant", "anthropic.claude-instant-v1"]]

In [27]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str) -> str:
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn
            }
        },
    )

    return response

In [34]:
query = "What is the parental leave policy at Bob's Pizza?"

for model_id in claude_model_ids:
    model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
    response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
    generated_text = response['output']['text']
    citations = response["citations"]
    contexts = []
    for citation in citations:
        retrievedReferences = citation["retrievedReferences"]
        for reference in retrievedReferences:
            contexts.append(reference["content"]["text"])
    print(f"---------- Generated using {model_id[0]}:")
    pp.pprint(generated_text )
    print(f'---------- The citations for the response generated by {model_id[0]}:')
    pp.pprint(contexts)
    print()

---------- Generated using Claude 3 Sonnet:
("According to the search results, Bob's Pizza offers all employees 12 weeks "
 'of paid parental leave and an additional 12 weeks of unpaid parental leave '
 'per child, per year. This generous parental leave policy is mentioned as '
 'being offered by Bob, who is described as "a proud father of five."')
---------- The citations for the response generated by Claude 3 Sonnet:
[ 'We aim to retain and motivate employees by offering attractive but '
  'realistic career moves allowing them to develop their skills in the '
  "long-term. Given the importance Bob's Pizza puts on cultural diversity, "
  'employees who are interested in international assignments can be given the '
  'opportunity to work in different countries. The international dimension of '
  'the Group is used as a competitive advantage to retain and develop talented '
  "people. At Bob's Pizza, promotions are based on sustained performance from "
  'a results and behaviour standpo

In [35]:
# Test using a prompt it should not be able to answer. The model should return that it doesn't know.
query = "What is the sick leave policy at Bob's Pizza?"

for model_id in claude_model_ids:
    model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
    response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
    generated_text = response['output']['text']
    citations = response["citations"]
    contexts = []
    for citation in citations:
        retrievedReferences = citation["retrievedReferences"]
        for reference in retrievedReferences:
            contexts.append(reference["content"]["text"])
    print(f"---------- Generated using {model_id[0]}:")
    pp.pprint(generated_text )
    print(f'---------- The citations for the response generated by {model_id[0]}:')
    pp.pprint(contexts)
    print()

---------- Generated using Claude 3 Sonnet:
('I could not find any information about a specific sick leave policy for '
 "employees at Bob's Pizza in the provided search results. The results discuss "
 "Bob's Pizza's policies and approaches around total rewards, employment and "
 'working conditions, training and learning, employee relations, and creating '
 'a flexible and dynamic organization, but do not mention details on sick '
 'leave.')
---------- The citations for the response generated by Claude 3 Sonnet:
[]

---------- Generated using Claude Instant:
("The search results do not contain any information about Bob's Pizza's "
 'specific sick leave policy.')
---------- The citations for the response generated by Claude Instant:
[]



### Run The Following To Clean Up Knowledge Base, OSS, S3 and IAM

In [36]:
# clean up Bedrock client
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

In [37]:
# clean up Knowledge Base
bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=kb['knowledgeBaseId'])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb['knowledgeBaseId'])
oss_client.indices.delete(index=index_name)
aoss_client.delete_collection(id=collection_id)
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

{'ResponseMetadata': {'RequestId': 'cd5f020e-d246-4d92-90f3-a03579e27488',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cd5f020e-d246-4d92-90f3-a03579e27488',
   'date': 'Wed, 05 Feb 2025 16:44:33 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [38]:
# clean up IAM
from utility import delete_iam_role_and_policies
delete_iam_role_and_policies()

0

In [ ]:
# clean up S3
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)

### If you no longer need the SageMaker notebook, remember to delete it. 